# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Load the csv into a data frame
weather_data = pd.read_csv("output/world_cities.csv")
# Check it
weather_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vendas Novas,38.6771,-8.4579,67.62,96,1,4.27,PT,1631772636
1,Arlit,18.7369,7.3853,89.85,17,5,6.15,NE,1631772637
2,Santa Cruz de la Sierra,-17.8000,-63.1667,63.75,40,34,10.18,BO,1631772639
3,Montepuez,-13.1256,38.9997,77.59,49,55,5.21,MZ,1631772641
4,Cockburn Town,21.4612,-71.1419,82.71,79,86,13.15,TC,1631772642


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [9]:
# Step 1: Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating = True, max_intensity = 100,
                                 point_radius = 20)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Step 1 - Find only cities that meet the following ideal weather conditions
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
ideal_weather_data = weather_data.loc[(weather_data["Max Temp"] < 80) & 
                                        (weather_data["Max Temp"] > 70) &
                                        (weather_data["Wind Speed"] < 10) & 
                                        (weather_data["Cloudiness"] == 0)]

# Step 2 - Drop any rows with null values
ideal_weather_data = ideal_weather_data.dropna()

# Step 3 - Select only the top 10 cities from this data
ideal_weather_data_top_10 = ideal_weather_data.head(10)
ideal_weather_data_top_10 # There are 10 rows

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,Caravelas,-17.7125,-39.2481,73.26,84,0,7.61,BR,1631772647
25,Janaúba,-15.8025,-43.3089,72.10,33,0,2.06,BR,1631772532
48,Jalu,29.0331,21.5482,74.70,44,0,7.05,LY,1631772737
129,Nālūt,30.3333,10.8500,74.52,28,0,8.75,LY,1631772844
186,Victor Harbor,-35.5667,138.6167,76.71,55,0,1.01,AU,1631772919
190,Sinnamary,5.3833,-52.9500,78.69,84,0,6.89,GF,1631772925
201,Itaueira,-7.6033,-43.0256,79.30,43,0,6.29,BR,1631772939
269,Saint-Pierre,-21.3393,55.4781,78.44,64,0,4.61,RE,1631773026
292,Zliten,32.4674,14.5687,75.31,67,0,9.48,LY,1631773061
409,Ginda,30.6945,78.4932,77.47,51,0,5.91,IN,1631773218


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = ideal_weather_data_top_10
hotel_df["Hotel Name"] = ""

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


<ipython-input-23-36895e6e2388>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_info
city_locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add layer
fig.add_layer(heat_layer)
markers = gmaps.marker_layer(city_locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))